In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
submission_example = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/submission.csv')
submission_example.head()

In [ ]:
test = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/test.csv')
test.head()
train = pd.read_csv('/kaggle/input/covid19-global-forecasting-week-2/train.csv')
train.head()

In [ ]:
train_by_country = train.groupby('Country_Region')

In [ ]:
train_by_country = train.groupby(["Country_Region","Date"],as_index= False).sum()
train_by_country.head()

In [ ]:
train["Date"] = pd.to_datetime(train["Date"])
test["Date"] = pd.to_datetime(test["Date"])
train["Day"] = pd.to_datetime(train["Date"]).dt.strftime("%m%d").astype(int) 
train['Day'] -= 122 #because first_date = 122
test["Day"] = pd.to_datetime(test["Date"]).dt.strftime("%m%d").astype(int) 
test['Day'] -= 122

In [ ]:
train.info()

In [ ]:
train.head()

In [ ]:
train['Province_State'].isna().sum()

In [ ]:
train.shape

In [ ]:
train[train['Country_Region']=='China']['Province_State'].unique()

In [ ]:
hk = train[train['Province_State']=='Hong Kong']
f, axarr = plt.subplots(2, sharex=True,figsize=(15,15))
f.suptitle('Hong Kong')
axarr[0].plot(hk['Date'], hk['ConfirmedCases'])
axarr[0].set(ylabel='Confirmed Cases')
axarr[1].plot(hk['Date'],hk['Fatalities'])
axarr[1].set(ylabel='Fatalities')
plt.xticks(rotation = 'vertical')
plt.show()


In [ ]:
train['Province_State'].fillna(train['Country_Region'],inplace = True)
test['Province_State'].fillna(test['Country_Region'],inplace = True)

In [ ]:
from sklearn import preprocessing
le1 = preprocessing.LabelEncoder()
le2 = preprocessing.LabelEncoder()

train['Country_Region_le'] = le1.fit_transform(train.Country_Region)
train['Province_State_le'] = le2.fit_transform(train.Province_State)
test['Country_Region_le'] = le1.fit_transform(test.Country_Region)
test['Province_State_le'] = le2.fit_transform(test.Province_State)

In [ ]:
train.head()

In [ ]:
#open_period only_use data 1/22~3/19
val = train[train["Day"] > 196].reset_index(drop = True)
train = train[train["Day"] <= 196].reset_index(drop = True)

In [ ]:
x_train = train.drop(['Id', 'ConfirmedCases','Fatalities','Date','Province_State','Country_Region'], axis = 1)
y_train_1 = train["ConfirmedCases"]
y_train_2 = train["Fatalities"]
x_val = val.drop(['Id', 'ConfirmedCases','Fatalities','Date','Province_State','Country_Region'], axis = 1)
y_val_1 = val["ConfirmedCases"]
y_val_2 = val["Fatalities"]
train.shape , val.shape

In [ ]:
def evaluate(predict,actual):                                              
    return np.sqrt(np.mean(np.square(np.log(predict+1) - np.log(actual+1))))

In [ ]:
import lightgbm as lgb
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor

In [ ]:

lgb_params = {
   'objective': 'regression',
    'metric': 'rmse',
}
lgb_train_1 = lgb.Dataset(x_train, y_train_1)
lgb_eval_1 = lgb.Dataset(x_val, y_val_1, reference=lgb_train_1)

lgb_model_1 = lgb.train(lgb_params,lgb_train_1, valid_sets=lgb_eval_1,num_boost_round=10000)
y_pred_1 = lgb_model_1.predict(x_val, num_iteration = lgb_model_1.best_iteration)

lgb_train_2 = lgb.Dataset(x_train, y_train_2)
lgb_eval_2 = lgb.Dataset(x_val, y_val_2, reference=lgb_train_2)

lgb_model_2 = lgb.train(lgb_params,lgb_train_2, valid_sets=lgb_eval_2,num_boost_round=5000)
y_pred_2 = lgb_model_1.predict(x_val, num_iteration = lgb_model_2.best_iteration)


In [ ]:
print('Light LGB')
print('Case Error:',evaluate(y_pred_1,y_val_1),'\nFatalties Error:', evaluate(y_pred_2,y_val_2),'\nAverage Error:',(evaluate(y_pred_1,y_val_1)+evaluate(y_pred_2,y_val_2))/2)

In [ ]:
xgb_model_1= XGBRegressor(n_estimators = 4000)
xgb_model_1.fit(x_train,y_train_1)
y_pred_1 = xgb_model_1.predict(x_val)
xgb_model_2= XGBRegressor(n_estimators = 4000)
xgb_model_2.fit(x_train,y_train_2)
y_pred_2 = xgb_model_2.predict(x_val)
print('XGB:')
print('Case Error:',evaluate(y_pred_1,y_val_1),'\nFatalties Error:', evaluate(y_pred_2,y_val_2),'\nAverage Error:',(evaluate(y_pred_1,y_val_1)+evaluate(y_pred_2,y_val_2))/2)

In [ ]:
XGB_val_result = pd.concat([x_val,pd.Series(y_pred_1),pd.Series(y_val_1),pd.Series(y_pred_2),pd.Series(y_val_2)],axis = 1)
XGB_val_result = XGB_val_result.rename(columns = {0:'y_pred_1',1:'y_pred_2'})

In [ ]:
XGB_val_result.loc[XGB_val_result['y_pred_1'] < 0, 'y_pred_1'] = 0 
XGB_val_result.loc[XGB_val_result['y_pred_2'] < 0, 'y_pred_2'] = 0 

In [ ]:
print('XGB:')
print('Case Error:',evaluate(XGB_val_result['y_pred_1'],y_val_1),'\nFatalties Error:', evaluate(XGB_val_result['y_pred_2'],y_val_2),'\nAverage Error:',(evaluate(y_pred_1,y_val_1)+evaluate(y_pred_2,y_val_2))/2)

In [ ]:
XGB_val_result['Case_pred_diff'] = np.sqrt((np.log(XGB_val_result['y_pred_1']+1) - np.log(XGB_val_result['ConfirmedCases'] + 1))**2)
XGB_val_result['Death_pred_diff'] = np.sqrt((np.log(XGB_val_result['y_pred_2']+1) - np.log(XGB_val_result['Fatalities'] + 1))**2)
XGB_val_result['Country_Region'] = le1.inverse_transform(XGB_val_result['Country_Region_le'])
XGB_val_result['Province_Region'] = le2.inverse_transform(XGB_val_result['Province_State_le'])


In [ ]:
XGB_val_result[XGB_val_result['Country_Region']=='US'].sort_values(by='Case_pred_diff',ascending = False).head(50)

In [ ]:
XGB_val_result.sort_values(by='Case_pred_diff', ascending = False).head(50)


In [ ]:
random_forest_1=RandomForestRegressor(bootstrap=True, 
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=15,
            min_weight_fraction_leaf=0.0, n_estimators=150, 
            random_state=0, verbose=0, warm_start=False)
random_forest_1.fit(x_train,y_train_1)
y_pred_1 = random_forest_1.predict(x_val)

random_forest_2=RandomForestRegressor(bootstrap=True, 
            max_depth=25, max_features='auto', max_leaf_nodes=None,
            min_samples_leaf=1, min_samples_split=15,
            min_weight_fraction_leaf=0.0, n_estimators=150, 
            random_state=0, verbose=0, warm_start=False)
random_forest_2.fit(x_train,y_train_2)
y_pred_2 = random_forest_2.predict(x_val)
print('Random Forest Regressor')
print('Case Error:',evaluate(y_pred_1,y_val_1),'\nFatalties Error:', evaluate(y_pred_2,y_val_2),'\nAverage Error:',(evaluate(y_pred_1,y_val_1)+evaluate(y_pred_2,y_val_2))/2)

In [ ]:
x_train

In [ ]:
random_forest_1.feature_importances_

In [ ]:
! pip install fbprophet

In [ ]:
hk_cases = hk[['Date','ConfirmedCases']]
hk_cases = hk_cases.rename(columns = {'Date':'ds','ConfirmedCases':'y'})
#hk_cases['y'] = hk_cases['y'].diff()
hk_cases.tail()

In [ ]:
from fbprophet import Prophet

m = Prophet()
m.fit(hk_cases)

In [ ]:
future = m.make_future_dataframe(periods = 90)
forecast = m.predict(future)
forecast.tail()

In [ ]:
fig1 = m.plot(forecast)

In [ ]:
sns.lineplot(data = hk_cases, x = 'ds',y = 'y')

In [ ]:
hk['diff_case'] = hk['ConfirmedCases'].diff()
hk['diff_fatal'] = hk['Fatalities'].diff()

hk.tail()

In [ ]:
train.head()

In [ ]:
ca = train[train['Province_State']=='California']
ca['diff_case'] = ca['ConfirmedCases'].diff()
ca['diff_fatal'] = ca['Fatalities'].diff()

ca.tail()

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(data = ca, x = 'Date',y = 'ConfirmedCases')
plt.title('California')
plt.xticks(rotation = 'vertical')
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
sns.lineplot(data = ca, x = 'Date',y = 'diff_case')
plt.title('California')
plt.xticks(rotation = 'vertical')
plt.show()

In [ ]:
ny = train[train['Province_State']=='New York']
ny['diff_case'] = ny['ConfirmedCases'].diff()
ny['diff_fatal'] = ny['Fatalities'].diff()

plt.figure(figsize=(20,10))
sns.lineplot(data = ny, x = 'Date',y = 'diff_case')
plt.title('New York')
plt.xticks(rotation = 'vertical')
plt.show()

In [ ]:
us = train[train['Country_Region']=='US']
us_cases = pd.DataFrame(columns = ('Date', 'Cases','Diff'))
for s in us['Province_State'].unique():
    curr = us[us['Province_State']==s]
    curr = curr[['Date','Province_State','ConfirmedCases']]
    curr.columns = ['Date','Province_State','Cases']
    curr['Diff'] = curr['Cases'].diff()
    us_cases = us_cases.append(curr)

In [ ]:
us_cases_sum = us_cases.groupby('Province_State').sum()
top10 = us_cases_sum.sort_values('Cases', ascending = False).head(10).index


In [ ]:
plt.figure(figsize=(20,10))
ax = sns.lineplot(data = us_cases[(us_cases['Date']>='2020-03-01') & (us_cases['Province_State'].isin(top10))], x = 'Date',y = 'Cases', hue = 'Province_State')
plt.title('US Confirmed Cases: Total')
ax.legend(loc=2)
plt.xticks(rotation = 45)
plt.show()

In [ ]:
plt.figure(figsize=(20,10))
ax = sns.lineplot(data = us_cases[(us_cases['Date']>='2020-03-01') & (us_cases['Province_State'].isin(top10[0:]))], x = 'Date',y = 'Diff', hue = 'Province_State')
plt.title('US Confirmed Cases Daily New Cases')
ax.legend(loc=2)
plt.xticks(rotation = 45)
plt.show()

In [ ]:
hk['diff_case'] = hk['ConfirmedCases'].diff()

In [ ]:
plt.figure(figsize=(20,8))
sns.lineplot(x = 'Date', y = 'diff_case', data = hk)
plt.xticks(rotation = 45)
plt.title('Daily New Confirmed COVID-19 Cases in Hong Kong')

In [ ]:
pred_result = pd.DataFrame({'Id': [], 'ConfirmedCases': [], 'Fatalities': [], 'y1_real':[], 'y2_real':[]})
for country in train['Country_Region'].unique():
    states = train[train['Country_Region']==country]['Province_State'].unique()
    for state in states:
        train_CS = train[(train['Country_Region']==country) & (train['Province_State']==state)]
        X_train_CS = train_CS[['Province_State_le', 'Country_Region_le', 'Day']]
        y1_train_CS = train_CS['ConfirmedCases']
        y2_train_CS = train_CS['Fatalities']
        
        val_CS = val[(val['Country_Region']==country) & (val['Province_State']==state)]
        X_val_CS = val_CS[['Province_State_le', 'Country_Region_le', 'Day']]
        y1_val_CS = val_CS['ConfirmedCases']
        y2_val_CS = val_CS['Fatalities']
        
        model1 = XGBRegressor(n_estimators=1000)
        model1.fit(X_train_CS, y1_train_CS)
        y1_pred = model1.predict(X_val_CS)
        
        model2 = XGBRegressor(n_estimators=1000)
        model2.fit(X_train_CS, y2_train_CS)
        y2_pred = model2.predict(X_val_CS)
        
        CS_result = pd.DataFrame({'Id': val_CS['Id'], 'ConfirmedCases': y1_pred, 'Fatalities': y2_pred,'y1_real':y1_val_CS, 'y2_real':y2_val_CS})
        pred_result = pd.concat([pred_result, CS_result], axis = 0)
        
        

In [ ]:
pred_result

In [ ]:
evaluate(pred_result['ConfirmedCases'], pred_result['y1_real'])

In [ ]:
evaluate(pred_result['Fatalities'], pred_result['y2_real'])

In [ ]:
pred_result['Case_pred_diff'] = np.sqrt((np.log(pred_result['y1_real']+1) - np.log(pred_result['ConfirmedCases'] + 1))**2)
pred_result['Death_pred_diff'] = np.sqrt((np.log(pred_result['y2_real']+1) - np.log(pred_result['Fatalities'] + 1))**2)


In [ ]:
pred_result.sort_values(by='Case_pred_diff', ascending = False).head(100)

In [ ]:
val.iloc[3223]

In [ ]:
val[val['Province_State']=='Michigan']['ConfirmedCases'].plot()

In [ ]:
train[train['Province_State']=='Michigan']

In [ ]:
train['From'] = 'train'
val['From'] = 'val'
all_data = pd.concat([train,val])

In [ ]:
michigan = all_data[all_data['Province_State']=='Michigan']
michigan['diff_case'] = michigan['ConfirmedCases'].diff()
plt.figure(figsize=(15,10))
sns.lineplot(x = 'Date',y = 'diff_case',hue = 'From',data = michigan)
plt.xticks(rotation = 45)
plt.title('Michigan')

In [ ]:
from statsmodels.tsa.arima_model import ARIMA

# 1,1,1ARIMA Model
model = ARIMA(michigan[michigan['From']=='train']['ConfirmedCases'], order=(1,1,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
michigan.columns

In [ ]:
# Build Model
# model = ARIMA(train, order=(3,2,1)) 
from statsmodels.tsa.arima_model import ARIMA

# 1,1,1ARIMA Model
michigan_train = michigan[michigan['From']=='train']['ConfirmedCases']
michigan_val = michigan[michigan['From']=='val']['ConfirmedCases']

model = ARIMA(michigan_train, order=(1, 1, 1))  
fitted = model.fit(disp=-1)  

# Forecast
fc, se, conf = fitted.forecast(13, alpha=0.05)  # 95% conf

# Make as pandas series
fc_series = pd.Series(fc, index=michigan[michigan['From']=='val']['Date'])
lower_series = pd.Series(conf[:, 0], index=michigan[michigan['From']=='val']['Date'])
upper_series = pd.Series(conf[:, 1], index=michigan[michigan['From']=='val']['Date'])

michigan_val = michigan[michigan['From']=='val']
michigan_val = michigan_val[['Date','ConfirmedCases']]
michigan_val.index = michigan_val['Date']

michigan_train = michigan[michigan['From']=='train']
michigan_train = michigan_train[['Date','ConfirmedCases']]
michigan_train.index = michigan_train['Date']

# Plot
plt.figure(figsize=(12,5), dpi=100)
plt.plot(michigan_train['ConfirmedCases'], label='training')

plt.plot(michigan_val['ConfirmedCases'], label='actual')
plt.plot(fc_series, label='forecast')
plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
plt.title('Michigan: Forecast vs Actuals')
plt.xticks(rotation = 45)
plt.legend(loc='upper left', fontsize=8)
plt.show()

In [ ]:
# 1,1,1ARIMA Model

us = all_data[all_data['Country_Region']=='US']

us_train = us[us['From']=='train']
us_val = us[us['From']=='val']

states = us['Province_State'].unique()

for state in states:
    print(state)
    
    cs_train = us_train[us_train['Province_State'] == state]
    cs_val = us_val[us_val['Province_State'] == state]
    if cs_train.empty:
        continue;
    if cs_val.empty:
        continue;
    if state in ['West Virginia']:
        continue
    model = ARIMA(cs_train['ConfirmedCases'], order=(1, 1, 1))  
    try:
        fitted = model.fit(disp=0)  
    except ValueError or LinAlgError :
        continue
    # Forecast
    fc, se, conf = fitted.forecast(len(cs_val), alpha=0.05)  # 95% conf
    
    # Make as pandas series
    fc_series = pd.Series(fc, index=cs_val['Date'])
    lower_series = pd.Series(conf[:, 0], index=cs_val['Date'])
    upper_series = pd.Series(conf[:, 1], index=cs_val['Date'])

    
    cs_val = cs_val[['Date','ConfirmedCases']]
    cs_val.index = cs_val['Date']

    
    cs_train = cs_train[['Date','ConfirmedCases']]
    cs_train.index = cs_train['Date']

    # Plot
    plt.figure(figsize=(12,5), dpi=100)
    plt.plot(cs_train['ConfirmedCases'], label='training')

    plt.plot(cs_val['ConfirmedCases'], label='actual')
    plt.plot(fc_series, label='forecast')
    plt.fill_between(lower_series.index, lower_series, upper_series, 
                 color='k', alpha=.15)
    plt.title(state+': Forecast vs Actuals')
    plt.xticks(rotation = 45)
    plt.legend(loc='upper left', fontsize=8)
    plt.show()

In [ ]:
cs_train['ConfirmedCases']

In [ ]:
us = all_data[all_data['Country_Region']=='US']

us_train = us[us['From']=='train']
us_val = us[us['From']=='val']

states = us['Province_State'].unique()

state = 'New York'
ny = us[us['Province_State']=='New York']
sns.lineplot(x = 'Date', y = 'ConfirmedCases', data = ny, hue = 'From') 
plt.title('New York')
plt.xticks(rotation = 45)

In [ ]:
us = train[train['Country_Region']=='US']
us = pd.merge(us, )

In [ ]:
us.head(15)

In [ ]:
us = train[train['Country_Region']=='US']
us_age_dist1=pd.read_csv('../input/us-age-dist1/us_age_dist.csv')
us = pd.merge(us, us_age_dist1)
us.head()